In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [19]:
# 2910 reconfinement
nn=1
JE,JI=3.60,3.2
a,b=0.0,0.0#1.5,0.25#0.3,0.2#
# a=-JI*(JE/b)#+0.8
# a=-JI/(JE/b)#-0.3

'''
J=np.zeros((nn*2,nn*2))
J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn
print('2by2 matrix:',J)
lvec,sv,rvech=la.svd(J)
print('sum:',np.sum(sv),'diff:',sv[0]-sv[1])
s1=np.sqrt(2*((JE+b)**2+(JI-a)**2))
s2=np.sqrt(2*((JE-b)**2+(JI+a)**2))
print('sum1:',s1,'; sum2:',s2)
'''

# generate perturbed matrix
nn=600
# # JE,JI=0.8,0.6
# # a,b=0.80,0.2#1.5,-0.3
# a1=-JE*b/JI
# a2=-JE*JI/b
# print('a1-',a1,' a2-',a2)
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print(' random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
x_= x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]
# print('theoretical solution:',theo_sv,'; numerical solution:',sv_[:2])

rholimit=1/svr[0]+svr[0]
derivphi=1/(1-svr[0]**2)
overlap=-1/derivphi/svr[0]**2
# print('am theo:',1/sv[0]**2)
print('overlap:',overlap)
print('overlap num:',np.sum(lvec_[:,0]*lvec[:,0])**2)

## calculate whether there is an additive relationship
overlap_num=1.0-1/svr[0]**2
alpha = np.sqrt(overlap_num)
lvar=lvec_[:,0]-alpha*lvec[:,0]
# plt.figure()
# n, bins, patches = plt.hist(lvar[:nn], 100, density=True, facecolor='g', alpha=0.75)

rvec_,rvec=rvech_.T,rvech.T
rvar=rvec_[:,0]-alpha*rvec[:,0]
# plt.figure()
# n, bins, patches = plt.hist(rvar[:nn], 100, density=True, facecolor='g', alpha=0.75)

#tilte overlap
tilteuv=np.sum(rvec_[:,0]*lvec_[:,0])
orguv=alpha**2*np.sum(rvec[:,0]*lvec[:,0])
cosuv=(1/svr[0]**2)*np.sum(rvec[:,0]*lvec[:,0])


# Tomorrow, check whether the overlap of the perturbed matrix is equal to the original * alpha square.

 random singular value: [6.81175455e+00 5.88095309e-14]
overlap: 0.9784482758620688
overlap num: 0.9787049714554014


Contents
* Can the perturbed matrix $\widetilde{X}$ be represented by basis vectors of low-rank matrix?
* Will the difference between different representations change with the properties of low-rank structure/random matrix (infimum/supramum)
* Can we use either of the (M,N)/(m,n) to represent the perturbed matrix?
* Now we use E/I structure (known already), how about change to use Gaussian distribution?

Edit by Yuxiu

30/10/2020 the $1^{st}$ version V0


In [37]:
# limit of the singular value of the perturbed matrix
# find the largest singular value which are larger than the critical value.
thetac = 2.0
rlarger=svr[np.where(svr>thetac)]
lenc = len(rlarger)
rhop=np.zeros(lenc)
x_tilde_rec=np.zeros_like(x_)
# reconstruct perturbed matrix using basis vectors of low-rank structure
for i in range(lenc):
    rhop[i]=1/svr[i]+svr[i]
    vecoverlap=1-1/svr[i]**2
    lvecrec=np.sqrt(vecoverlap)*lvec[:,i]
    rvecrec=np.sqrt(vecoverlap)*rvec[:,i]
    x_tilde_rec +=rhop[i]*np.reshape(lvecrec,(2*nn,1))@np.reshape(rvecrec,(1,nn*2))#rhop[i]*(lvecrec@rvecrec.T)
print(lenc)


errmat=x_-x_tilde_rec#(rhop[0]*np.reshape(lvecrec,(nn*2,1))@np.reshape(rvecrec,(1,nn*2)))#(sv_[0]*np.reshape(lvec_[:,0],(nn*2,1))@np.reshape(rvec_[:,0].T,(1,nn*2)))
# check the singular value of the Error-matrix
lvecrec,svrec,rvechrec=la.svd(x_tilde_rec)
lvecerr,sverr,rvecherr=la.svd(errmat)
print('recon svrho:',svrec[:5],'; num svrho:',svr[:5])
print('num vec overlap:',np.sum(rvec_[:,0]*rvec[:,0])**2,' theo overlap:',vecoverlap)

print('singular values (largest) of the error matrix:',sverr[:5],sv_[:5])
# fig=plt.figure()
# ax=fig.add_subplot(111)
# n, bins, patches = ax.hist(sverr, 50, density=True, facecolor='g', alpha=0.75)


1
recon svrho: [6.80859064e+00 6.63107729e-14 6.31349745e-14 5.42914578e-14
 5.31285661e-14] ; num svrho: [6.81175455e+00 5.88095309e-14 5.73088770e-14 4.43769051e-14
 4.24557572e-14]
num vec overlap: 0.9788613601962554  theo overlap: 0.9784482758620688
singular values (largest) of the error matrix: [1.98798809 1.97500857 1.96817541 1.96427662 1.95175516] [6.91701634 1.987878   1.974473   1.96742004 1.96348708]


In [41]:
# check whether the Generalized SVD can identically represent the perturbed matrix (all the properties)
# we start from rank-1 perturbation 
m,n=np.ones((nn*2,1)),np.zeros((nn*2,1))
n[:nn,0],n[nn:,0]=JE,-JI
newrep=(1-1/svr[0]**4)*m@n.T/nn
diff_rep=newrep-x_tilde_rec
# plt.imshow(diff_rep)
newrandMat=x_-newrep
lvecnr,svnr,rvechnr=la.svd(newrandMat)
fig=plt.figure()
ax=fig.add_subplot(121)
n, bins, patches = ax.hist(svnr, 50, density=True, facecolor='g', alpha=0.75)

lvecr,svr,rvechr=la.svd(x)
ax1=fig.add_subplot(122)
n, bins, patches = ax1.hist(svr, 50, density=True, facecolor='g', alpha=0.75)

In [ ]:
# some simple analysis, how rho_1 changes with JE and JI, how the overlap of vectors changes with JE and JI


In [58]:
# Generate Gaussian vector of E neuron
JEmI,JEpI=-0.5,2.4
nn,a,b=1000,0.6,-0.6
yE,yI= np.random.randn(nn),np.random.randn(nn)


JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
print(JE,JI)
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=yE,-yE
N2[:nn],N2[nn:]=a*yE/nn,b*yE/nn
print(np.mean(a*yE),np.mean(b*yI))

print('overlap M2N2:',np.sum(M2*N2),'; M1N2:',np.sum(M1*N2))

# Jr=np.zeros((nn*2,nn*2))
# Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
# lvec,svr,rvech=la.svd(Jr)


0.95 1.45
0.04471048182729902 0.030153174468154503
overlap M2N2: 1.2959560012092206 ; M1N2: 0.0


In [59]:
J = np.reshape(M1,(2*nn,1))@np.reshape(N1,(1,2*nn))
J +=np.reshape(M2,(2*nn,1))@np.reshape(N2,(1,2*nn))
print(np.shape(J))

(2000, 2000)


In [75]:
def tanhpfunc(x):
    return np.tanh(x)+1.0
def dynxact(J,x0,timeparams,type='tanhp'):
    tfinal,dt=timeparams[0],timeparams[1]
    nn=np.shape(J)[0]
    nt = int(tfinal/dt)
    xactdyn=np.zeros((nn,nt))
    # print(xFP)
    xactdyn[:,0]=x0
    eps=1e-5
    if type=='tanhp':
        for it in range(1,nt):
            phisyn=np.reshape(tanhpfunc(xactdyn[:,it-1]),(nn,1))
            deltaxact=dt*(-xactdyn[:,it-1]+np.squeeze(J@phisyn))
            xactdyn[:,it]=xactdyn[:,it-1]+deltaxact
    return xactdyn
timeparams=np.array([100,0.1])
x0=-np.random.rand(nn*2)
x0[nn:]-=1.0


xact=dynxact(J,x0,timeparams,type='tanhp')

In [76]:
# plt.figure()
tt = np.linspace(0,100,1000)

# for i in range(30):
#     plt.plot(tt,xact[i,:],'r')
#     plt.plot(tt,xact[nn+i,:],'b')
frE,frI = xact[:nn,-1],xact[nn:,-1]


In [77]:
fig=plt.figure()
ax=fig.add_subplot(211)
n, bins, patches = ax.hist(frE, 50, density=True, facecolor='g', alpha=0.75)
ax1=fig.add_subplot(212)
n, bins, patches = ax1.hist(frI, 50, density=True, facecolor='g', alpha=0.75)